In [1]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import pandas_profiling as pp
import pandasql as ps
import datetime
import time

In [2]:
# Specifying the column name and its type in the form of a dictionary to be used later in the code
dtypes = {'ID':'int32',
          'Month_End_D':'str',
          'Customer_ID':'int32',
          'Region':'category',
          'LOB':'category',
          'INDUSTRY_CODE':'category',
          'Monthly_Revenue':'str',
          'Rolling12_Revenue':'str',
          'TENURE_IN_MONTHS':'str',
          'DEPOSIT_ACCT_CNT':'str',
          'GL_BAL_S':'str',
          'LOAN_ACCT_CNT':'str',
          'LOAN_BAL_S':'str',
          'SRVC_ACCT_CNT':'str',
          'SRVC_ACCT_TRNS_CNT':'str',
          'MIN_PRMRY_CUST_OPEN_D':'str',
          'MAX_PRMRY_CUST_CLSD_D':'str',
         }

pd.options.display.float_format = '{:.2f}'.format

# Reading the csv file directly using read_csv
start_time=time.clock()
print("Start Time:", start_time)
data=pd.read_csv('CMU_Data_AllMonths.txt', sep='|', dtype=dtypes)
end_time=time.clock()
print("End Time:", end_time)

print("Execution Time:", end_time-start_time," seconds\n")
print("\nshape of the data is:", data.shape,"\n")

data.info()

Start Time: 2.104198
End Time: 23.018244
Execution Time: 20.914046  seconds


shape of the data is: (7983742, 17) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7983742 entries, 0 to 7983741
Data columns (total 17 columns):
ID                       int32
Month_End_D              object
Customer_ID              int32
Region                   category
LOB                      category
INDUSTRY_CODE            category
Monthly_Revenue          object
Rolling12_Revenue        object
TENURE_IN_MONTHS         object
DEPOSIT_ACCT_CNT         object
GL_BAL_S                 object
LOAN_ACCT_CNT            object
LOAN_BAL_S               object
SRVC_ACCT_CNT            object
SRVC_ACCT_TRNS_CNT       object
MIN_PRMRY_CUST_OPEN_D    object
MAX_PRMRY_CUST_CLSD_D    object
dtypes: category(3), int32(2), object(12)
memory usage: 814.7+ MB


In [3]:
start_time=time.clock()
print("Start Time: ",start_time,"\n")

# Drop ID column as it is not useful for the Data Analysis part
data.drop(columns='ID',inplace=True)

# Convert String Columns to Float
data['Monthly_Revenue'] = pd.to_numeric(data['Monthly_Revenue'], errors='coerce')
data['Rolling12_Revenue'] = pd.to_numeric(data['Rolling12_Revenue'], errors='coerce')
data['GL_BAL_S'] = pd.to_numeric(data['GL_BAL_S'], errors='coerce')
data['LOAN_BAL_S'] = pd.to_numeric(data['LOAN_BAL_S'], errors='coerce')

# Convert String Columns to Int
data['TENURE_IN_MONTHS'] = pd.to_numeric(data['TENURE_IN_MONTHS'], errors='coerce', downcast='integer').fillna(0)
data['TENURE_IN_MONTHS'] = data['TENURE_IN_MONTHS'].astype(int)

data['DEPOSIT_ACCT_CNT'] = pd.to_numeric(data['DEPOSIT_ACCT_CNT'], errors='coerce', downcast='integer').fillna(0)
data['DEPOSIT_ACCT_CNT'] = data['DEPOSIT_ACCT_CNT'].astype(int)

data['LOAN_ACCT_CNT'] = pd.to_numeric(data['LOAN_ACCT_CNT'], errors='coerce', downcast='integer').fillna(0)
data['LOAN_ACCT_CNT'] = data['LOAN_ACCT_CNT'].astype(int)

data['SRVC_ACCT_CNT'] = pd.to_numeric(data['SRVC_ACCT_CNT'], errors='coerce', downcast='integer').fillna(0)
data['SRVC_ACCT_CNT'] = data['SRVC_ACCT_CNT'].astype(int)

data['SRVC_ACCT_TRNS_CNT'] = pd.to_numeric(data['SRVC_ACCT_TRNS_CNT'], errors='coerce', downcast='integer').fillna(0)
data['SRVC_ACCT_TRNS_CNT'] = data['SRVC_ACCT_TRNS_CNT'].astype(int)

# Transform String to Dates
data['MIN_PRMRY_CUST_OPEN_D'] = data.MIN_PRMRY_CUST_OPEN_D.fillna('01JAN1801')
data['MIN_PRMRY_CUST_OPEN_D'].replace('','01JAN1801',inplace=True)
data['MIN_PRMRY_CUST_OPEN_D'] = pd.to_datetime(data['MIN_PRMRY_CUST_OPEN_D'], format='%d%b%Y')
# data['MIN_PRMRY_CUST_OPEN_D'] = data['MIN_PRMRY_CUST_OPEN_D'].dt.strftime('%Y-%m')

data['Month_End_D'] = pd.to_datetime(data['Month_End_D'], format='%d%b%Y')
# data['Month_End_D'] = data['Month_End_D'].dt.strftime('%Y-%m')

data['MAX_PRMRY_CUST_CLSD_D'] = data.MAX_PRMRY_CUST_CLSD_D.fillna('31DEC2100')
data['MAX_PRMRY_CUST_CLSD_D'].replace('','31DEC2100',inplace=True)
data['MAX_PRMRY_CUST_CLSD_D'] = data['MAX_PRMRY_CUST_CLSD_D'].str.replace('9999','2100')
data['MAX_PRMRY_CUST_CLSD_D'] = pd.to_datetime(data['MAX_PRMRY_CUST_CLSD_D'], format='%d%b%Y')
# data['MAX_PRMRY_CUST_CLSD_D'] = data['MAX_PRMRY_CUST_CLSD_D'].dt.strftime('%Y-%m')

end_time=time.clock()
print("End Time: ",end_time)
print("Execution Time:", end_time-start_time," seconds\n")

Start Time:  23.176853 

End Time:  137.458264
Execution Time: 114.28141100000002  seconds



In [4]:
data.head(5)

,Month_End_D,Customer_ID,Region,LOB,INDUSTRY_CODE,Monthly_Revenue,Rolling12_Revenue,TENURE_IN_MONTHS,DEPOSIT_ACCT_CNT,GL_BAL_S,LOAN_ACCT_CNT,LOAN_BAL_S,SRVC_ACCT_CNT,SRVC_ACCT_TRNS_CNT,MIN_PRMRY_CUST_OPEN_D,MAX_PRMRY_CUST_CLSD_D
0,2017-12-31,1,Region 1,None,Industry 1,nan,nan,562,29,230499.19,7243,732342015.48,537119,0,1971-02-15,2100-12-31
1,2019-01-31,1,Region 1,None,Industry 1,nan,nan,575,25,412883.71,6871,432884988.46,606733,21,1971-02-15,2100-12-31
2,2018-09-30,1,Region 1,None,Industry 1,nan,nan,571,27,627651.20,7276,403594567.27,574323,21,1971-02-15,2100-12-31
3,2018-01-31,1,Region 1,None,Industry 1,nan,nan,563,28,234071.10,7860,251859315.81,541082,0,1971-02-15,2100-12-31
4,2018-11-30,1,Region 1,None,Industry 1,nan,nan,573,24,734653.67,7126,273281907.59,592179,21,1971-02-15,2100-12-31


In [5]:
data.describe(include='all')

,Month_End_D,Customer_ID,Region,LOB,INDUSTRY_CODE,Monthly_Revenue,Rolling12_Revenue,TENURE_IN_MONTHS,DEPOSIT_ACCT_CNT,GL_BAL_S,LOAN_ACCT_CNT,LOAN_BAL_S,SRVC_ACCT_CNT,SRVC_ACCT_TRNS_CNT,MIN_PRMRY_CUST_OPEN_D,MAX_PRMRY_CUST_CLSD_D
count,7983742,7983742.00,7983742,7983742,7983742,7859267.00,2302632.00,7983742.00,7983742.00,7981970.00,7983742.00,7981970.00,7983742.00,7983742.00,7983742,7983742
unique,31,nan,56,7,25,nan,nan,nan,nan,nan,nan,nan,nan,nan,17048,1517
top,2017-04-30 00:00:00,nan,Region 1,LOB 1,Industry 1,nan,nan,nan,nan,nan,nan,nan,nan,nan,1978-01-01 00:00:00,2100-12-31 00:00:00
freq,270124,nan,1759438,7347032,7482385,nan,nan,nan,nan,nan,nan,nan,nan,nan,59992,7025351
first,2017-01-31 00:00:00,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,1801-01-01 00:00:00,1999-02-24 00:00:00
last,2019-07-31 00:00:00,nan,NaN,NaN,NaN,nan,nan,nan,nan,nan,nan,nan,nan,nan,2019-08-16 00:00:00,2100-12-31 00:00:00
mean,NaN,181723.80,NaN,NaN,NaN,1190.29,13654.94,172.06,1.67,104018.28,41.87,3013734.48,3335.39,22.98,NaN,NaN
std,NaN,104952.70,NaN,NaN,NaN,35712.47,301127.60,169.30,5.38,3311727.64,533.02,43003002.98,42811.00,1717.28,NaN,NaN
min,NaN,1.00,NaN,NaN,NaN,-44537654.32,-40572893.09,-20.00,0.00,-55345877.05,0.00,-1189010.06,0.00,0.00,NaN,NaN
25%,NaN,90800.00,NaN,NaN,NaN,10.56,112.65,45.00,1.00,406.88,0.00,0.00,0.00,0.00,NaN,NaN


In [6]:
start_time=time.clock()
print("Start Time: ",start_time,"\n")

print("The new shape of the original data:",data.shape)
print("Unique customers in original data:", data['Customer_ID'].nunique(),"\n")

df_2017=data[(data['Month_End_D'] > '2016-12-31') & (data['Month_End_D'] < '2018-01-01')]
print("The shape of the 2017 data:",df_2017.shape)
print("Unique customers in 2017 data:", df_2017['Customer_ID'].nunique(),"\n")

df_2018=data[(data['Month_End_D'] > '2017-12-31') & (data['Month_End_D'] < '2019-01-01')]
print("The shape of the 2018 data:",df_2018.shape)
print("Unique customers in 2018 data:", df_2018['Customer_ID'].nunique(),"\n")

df_2019=data[(data['Month_End_D'] > '2018-12-31') & (data['Month_End_D'] < '2020-01-01')]
print("The shape of the 2019 data:",df_2019.shape)
print("Unique customers in 2019 data:", df_2019['Customer_ID'].nunique(),"\n")

monthly_cohort=data[(data['Month_End_D'] >= '2017-12-31') & (data['Month_End_D'] <= '2019-01-31')]
print("The shape of the Cohort data:",monthly_cohort.shape)
print("Unique customers in Cohort data:", monthly_cohort['Customer_ID'].nunique(),"\n")
print("The columns in the cohort data are\n",monthly_cohort.columns)

end_time=time.clock()
print("End Time: ",end_time)
print("Execution Time:", end_time-start_time," seconds\n")

Start Time:  145.860836 

The new shape of the original data: (7983742, 16)
Unique customers in original data: 363481 

The shape of the 2017 data: (3121912, 16)
Unique customers in 2017 data: 308372 

The shape of the 2018 data: (3042093, 16)
Unique customers in 2018 data: 293658 

The shape of the 2019 data: (1819737, 16)
Unique customers in 2019 data: 282122 

The shape of the Cohort data: (3549353, 16)
Unique customers in Cohort data: 299061 

The columns in the cohort data are
 Index(['Month_End_D', 'Customer_ID', 'Region', 'LOB', 'INDUSTRY_CODE',
       'Monthly_Revenue', 'Rolling12_Revenue', 'TENURE_IN_MONTHS',
       'DEPOSIT_ACCT_CNT', 'GL_BAL_S', 'LOAN_ACCT_CNT', 'LOAN_BAL_S',
       'SRVC_ACCT_CNT', 'SRVC_ACCT_TRNS_CNT', 'MIN_PRMRY_CUST_OPEN_D',
       'MAX_PRMRY_CUST_CLSD_D'],
      dtype='object')
End Time:  148.905248
Execution Time: 3.044411999999994  seconds



In [7]:
start_time=time.clock()
print("Start Time: ",start_time,"\n")

# Date Manipulation for cohort analysis
c1=monthly_cohort
c1['mnth'] = c1['Month_End_D'].dt.strftime('%m').astype(int)
c1['year'] = c1['Month_End_D'].dt.strftime('%Y').astype(int)

c1['open_mnth'] = c1['MIN_PRMRY_CUST_OPEN_D'].dt.strftime('%m').astype(int)
c1['open_year'] = c1['MIN_PRMRY_CUST_OPEN_D'].dt.strftime('%Y').astype(int)

c1['close_mnth'] = c1['MAX_PRMRY_CUST_CLSD_D'].dt.strftime('%m').astype(int)
c1['close_year'] = c1['MAX_PRMRY_CUST_CLSD_D'].dt.strftime('%Y').astype(int)

print("The shape of the test data for cohort:",c1.shape)
print("Unique customers in the test data for cohort:", c1['Customer_ID'].nunique(),"\n")
print("The columns in the the test data for cohort are\n",c1.columns)

end_time=time.clock()
print("End Time: ",end_time)
print("Execution Time:", end_time-start_time," seconds\n")

Start Time:  148.930264 

The shape of the test data for cohort: (3549353, 22)
Unique customers in the test data for cohort: 299061 

The columns in the the test data for cohort are
 Index(['Month_End_D', 'Customer_ID', 'Region', 'LOB', 'INDUSTRY_CODE',
       'Monthly_Revenue', 'Rolling12_Revenue', 'TENURE_IN_MONTHS',
       'DEPOSIT_ACCT_CNT', 'GL_BAL_S', 'LOAN_ACCT_CNT', 'LOAN_BAL_S',
       'SRVC_ACCT_CNT', 'SRVC_ACCT_TRNS_CNT', 'MIN_PRMRY_CUST_OPEN_D',
       'MAX_PRMRY_CUST_CLSD_D', 'mnth', 'year', 'open_mnth', 'open_year',
       'close_mnth', 'close_year'],
      dtype='object')
End Time:  236.440078
Execution Time: 87.509814  seconds



In [8]:
def pysqldf(q):
    return ps.sqldf(q, globals())

In [13]:
q1 = """
SELECT strftime('%Y-%m', m1.Month_End_D) as month,
       count(distinct m1.Customer_ID) as total_customers,
       sum(case when m1.open_mnth = m1.mnth and m1.open_year = m1.year then 1 else 0 end) as new,
       sum(case when m1.close_mnth = m1.mnth and m1.close_year = m1.year then 1 else 0 end) as churned
FROM c1 m1
group by month;
"""
start_time=time.clock()
print("Start Time: ",start_time)
churn1=pysqldf(q1)
end_time=time.clock()
print("End Time: ",end_time)

print("Execution Time:", end_time-start_time," seconds\n")
churn1.set_index('month',inplace=True)

Start Time:  829.735877
End Time:  959.373254
Execution Time: 129.63737700000001  seconds



In [14]:
churn1

,total_customers,new,churned
month,,,
2017-12,254435,1431,1816
2018-01,254163,1737,2439
2018-02,252474,1533,1950
2018-03,252128,1942,1956
2018-04,252061,1739,2011
2018-05,251632,1899,2051
2018-06,251787,1769,1771
2018-07,254027,1570,1777
2018-08,261254,1873,2152


In [10]:
q2 = """
SELECT strftime('%Y-%m', a.Month_End_D) as month1,
       strftime('%Y-%m', b.Month_End_D) as month2,
       count(distinct b.Customer_ID) as count
FROM monthly_cohort a
LEFT JOIN monthly_cohort b on a.Customer_ID = b.Customer_ID and b.Month_End_D >= a.Month_End_D
WHERE strftime('%Y-%m', a.Month_End_D) <> strftime('%Y-%m', a.MIN_PRMRY_CUST_OPEN_D)
group by month1, month2;
"""
start_time=time.clock()
print("Start Time: ",start_time)
churn2=pysqldf(q2)
end_time=time.clock()
print("End Time: ",end_time)

churn2=pd.pivot_table(churn2,values='count',index='month1',columns=['month2'], fill_value = 0, aggfunc=sum)
print("Execution Time:", end_time-start_time," seconds\n")
churn2

Start Time:  370.308646
End Time:  570.99044
Execution Time: 200.68179400000002  seconds



month2,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01
month1,,,,,,,,,,,,,,
2017-12,253004,248523,244542,241362,237807,234822,232150,231004,233351,227066,224111,219780,219379,217629
2018-01,0,252426,247640,244231,240623,237541,234678,233502,235301,228899,225925,221526,221050,219387
2018-02,0,0,250941,246875,243057,239983,236928,235427,236388,230751,227695,223288,222805,221019
2018-03,0,0,0,250186,245534,242216,239103,237521,237892,232724,229640,225150,224601,222834
2018-04,0,0,0,0,250322,245113,241928,240245,240071,235307,232226,227655,227021,225232
2018-05,0,0,0,0,0,249733,245402,243180,242278,238028,234764,229996,229344,227364
2018-06,0,0,0,0,0,0,250018,246583,244891,241174,237837,232938,232220,230181
2018-07,0,0,0,0,0,0,0,252457,249599,246075,242674,237536,236850,234774
2018-08,0,0,0,0,0,0,0,0,259381,248474,245012,239850,239086,236976


In [11]:
q3 = """
SELECT strftime('%Y-%m', a.Month_End_D) as month1,
       strftime('%Y-%m', b.Month_End_D) as month2,
       count(distinct b.Customer_ID) as count
FROM monthly_cohort a
LEFT JOIN monthly_cohort b on a.Customer_ID = b.Customer_ID and b.Month_End_D >= a.Month_End_D
WHERE strftime('%Y-%m', a.Month_End_D) = strftime('%Y-%m', a.MIN_PRMRY_CUST_OPEN_D)
group by month1, month2;
"""
start_time=time.clock()
print("Start Time: ",start_time)
churn3=pysqldf(q3)
end_time=time.clock()
print("End Time: ",end_time)

churn3=pd.pivot_table(churn3,values='count',index='month1',columns=['month2'], fill_value = 0, aggfunc=sum)
print("Execution Time:", end_time-start_time," seconds\n")
churn3

Start Time:  571.031674
End Time:  701.418933
Execution Time: 130.38725900000009  seconds



month2,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01
month1,,,,,,,,,,,,,,
2017-12,1431,1367,1342,1288,1256,1227,1194,1155,1184,1112,1084,1065,1048,1027
2018-01,0,1737,1701,1651,1602,1570,1529,1500,1509,1430,1399,1370,1338,1324
2018-02,0,0,1533,1486,1425,1399,1373,1352,1369,1289,1267,1239,1221,1199
2018-03,0,0,0,1942,1879,1827,1799,1760,1764,1676,1658,1604,1569,1541
2018-04,0,0,0,0,1739,1686,1644,1610,1600,1543,1522,1471,1457,1429
2018-05,0,0,0,0,0,1899,1853,1802,1770,1695,1660,1612,1565,1527
2018-06,0,0,0,0,0,0,1769,1734,1689,1640,1606,1545,1509,1473
2018-07,0,0,0,0,0,0,0,1570,1515,1470,1448,1405,1374,1349
2018-08,0,0,0,0,0,0,0,0,1873,1826,1784,1726,1668,1636


In [15]:
q4 = """
SELECT strftime('%Y-%m', a.Month_End_D) as month1,
       strftime('%Y-%m', b.Month_End_D) as month2,
       count(distinct b.Customer_ID) as count
FROM monthly_cohort a
LEFT JOIN monthly_cohort b on a.Customer_ID = b.Customer_ID and b.Month_End_D >= a.Month_End_D
group by month1, month2;
"""
start_time=time.clock()
print("Start Time: ",start_time)
churn4=pysqldf(q4)
end_time=time.clock()
print("End Time: ",end_time)

churn4=pd.pivot_table(churn4,values='count',index='month1',columns=['month2'], fill_value = 0, aggfunc=sum)
print("Execution Time:", end_time-start_time," seconds\n")
churn4

Start Time:  959.400762
End Time:  1156.012174
Execution Time: 196.61141199999997  seconds



month2,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01
month1,,,,,,,,,,,,,,
2017-12,254435,249890,245884,242650,239063,236049,233344,232159,234535,228178,225195,220845,220427,218656
2018-01,0,254163,249341,245882,242225,239111,236207,235002,236810,230329,227324,222896,222388,220711
2018-02,0,0,252474,248361,244482,241382,238301,236779,237757,232040,228962,224527,224026,222218
2018-03,0,0,0,252128,247413,244043,240902,239281,239656,234400,231298,226754,226170,224375
2018-04,0,0,0,0,252061,246799,243572,241855,241671,236850,233748,229126,228478,226661
2018-05,0,0,0,0,0,251632,247255,244982,244048,239723,236424,231608,230909,228891
2018-06,0,0,0,0,0,0,251787,248317,246580,242814,239443,234483,233729,231654
2018-07,0,0,0,0,0,0,0,254027,251114,247545,244122,238941,238224,236123
2018-08,0,0,0,0,0,0,0,0,261254,250300,246796,241576,240754,238612
